In [ ]:
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import json

with open("outfit_path_list.json", "r") as f :
    outfit_path_list = json.load(f)
with open("top_bottom_path_list.json", "r") as f :
    top_bottom_path_list = json.load(f)

print(len(outfit_path_list))
print(len(top_bottom_path_list))

for p1_p2 in tqdm(top_bottom_path_list) :
    try :
        top_base_path, bottom_base_path = p1_p2.split(",")

        top_garment_split, _, top_garment_id = list(Path(top_base_path).parts)[-3:]
        TOP_PATH = os.path.join(
            "SAMPLE_DATA", top_garment_split, top_garment_id,
            f"{top_garment_id}__00__clo.json"
        )

        bottom_garment_split, _, bottom_garment_id = list(Path(bottom_base_path).parts)[-3:]
        BOTTOM_PATH = os.path.join(
            "SAMPLE_DATA", bottom_garment_split, bottom_garment_id,
            f"{bottom_garment_id}__00__clo.json"
        )

        # top_garment_split, top_garment_id = os.path.basename(TOP_PATH).split("__")[:-1]
        # bottom_garment_split, bottom_garment_id = os.path.basename(BOTTOM_PATH).split("__")[:-1]

        COMBINED_CLO_JSON_PATH = os.path.join(
            "SAMPLE_DATA", top_garment_split, 
            f"{top_garment_id}__00__{bottom_garment_id}__00",
            f"{top_garment_id}__00__{bottom_garment_id}__00__clo.json"
        )
        os.makedirs(os.path.dirname(COMBINED_CLO_JSON_PATH), exist_ok=True)

        with open(TOP_PATH, 'r') as file:
            top_json = json.load(file)

        with open(BOTTOM_PATH, 'r') as file:
            bottom_json = json.load(file)

        combined_clo_json = combine_top_bottom_clo_json(top_json, bottom_json)

        with open(COMBINED_CLO_JSON_PATH, 'w') as file:
            json.dump(combined_clo_json, file, indent=4)
    except Exception as e :
        print(e)
        print(p1_p2)
        print("="*100)
        continue

In [ ]:
import os
import json
import import_api
import utility_api
import fabric_api
import pattern_api
import export_api
import ApiTypes
import time
import os
import glob
import random
# definition
def open_json(path):
    with open(path[0], “r”) as f:
        data = json.load(f)
    return data
def garment_info_json(path):
    with open(path, “r”, encoding=“utf-8”) as file:
        info = json.load(file)
    return info
def seam_taping(stitch_info):
    for value in stitch_info:
        panel_num = value[‘panel’]
        edge_num = value[‘edge’]
        pattern_api.SetPatternPieceSeamtaping(panel_num, edge_num, True)
        pattern_api.SetPatternPieceSametapingWidth(panel_num, edge_num , 2)
def random_fabric_path(fabric_path):
    num1, num2 = random.sample(range(0, len(os.listdir(fabric_path)) - 1), 2)
    fabric_paths = [os.path.join(fabric_path, path) for path in os.listdir(fabric_path)]
    return fabric_paths[num1], fabric_paths[num2]
def adding_outfit_fabric(end_num_index):
    for num in range(0, end_num_index, 1):
        fabric_api.AssignFabricToPattern( 2, num, 2)
        pattern_api.SetArrangementShapeStyle(num, “Flat”)
def adding_top_bottom_fabric(end_num_index, end_num_index_2):
    for num in range(0, end_num_index, 1):
        fabric_api.AssignFabricToPattern( 2, num, 2)
        pattern_api.SetArrangementShapeStyle(num, “Flat”)
#        fabric_api.AssignFabricToPattern( 1, num, 1)
    for second_num in range(end_num_index, end_num_index + end_num_index_2, 1):
        fabric_api.AssignFabricToPattern( 3, second_num, 3)#2)
        pattern_api.SetArrangementShapeStyle(num, “Flat”)
ieo_option = ApiTypes.ImportExportOption()
ieo_option.bExportGarment = True
ieo_option.bExportAvatar = False
ieo_option.bSingleObject = True # single or Multiple
#ieo_option.weldType = WELD_TYPE
ieo_option.bThin = False
ieo_option.bSaveInZip = False
ieo_option.bMetaData = True
# ??? load
TYPE = [“outfit”, “top_bottom”]
#avtfile_path = “/Users/seph/Documents/clo/Assets/Avatar/Avatar/Male_V2/2025_0205_1806jinho.avt”
#avtfile_path = “/Users/seph/Documents/clo/Assets/Avatar/Avatar/Male_V2/2025_0209_jinho_?????????.avt”
#avtfile_path = r”D:\VTO2025\DATASETs\Ours1\CLO_JSON_DATAs\Mia_avt_revise.avt”
#avtfile_path = r”C:\Users\dytpq0916\Desktop\data_dir\174_65_miya_noac.avt”
avtfile_path = r”C:\Users\dytpq0916\Downloads\mia_173_55.avt”
custom_point_path = r”D:/VTO2025/DATASETs/Ours1/CLO_JSON_DATAs/View_Point”
pose_path = r”D:\VTO2025\DATASETs\Ours1\CLO_JSON_DATAs\Pose\A.pos”
DATA_DIR = r”D:\VTO2025\DATASETs\Ours1\CLO_JSON_DATAs”
FABRIC_PATH = r”D:\VTO2025\DATASETs\Ours1\CLO_JSON_DATAs\Filtered_Fabrics”
Garment_info_path = r”D:\VTO2025\DATASETs\Ours1\CLO_JSON_DATAs\info”
pass_garment_list = (“garments_5000_0”,“garments_5000_1",“garments_5000_10”,“garments_5000_11",“garments_5000_12”,“garments_5000_13",“garments_5000_20”,“garments_5000_21",“garments_5000_22")
for type in TYPE:
    DATA_TYPE_DIR = os.path.join(DATA_DIR, f”{type}_CLO_DATASET”) # outfit?? top_bottom?? ??
    for path in os.listdir(DATA_TYPE_DIR):
        # ?? texture root ????
        GARMENT_NUM_DIR = os.path.join(DATA_TYPE_DIR, path) # train test valid
        for garment_num in os.listdir(GARMENT_NUM_DIR):# garment_5000_?
            if garment_num.startswith(pass_garment_list):
                # didn’t need to do anything
                continue
            else:
                GARMENT_RAND_DIR = os.path.join(GARMENT_NUM_DIR, garment_num)
                for garment_dir in os.listdir(GARMENT_RAND_DIR): # rand_????_rand_????
                    GARMENT_DIR = os.path.join(GARMENT_RAND_DIR, garment_dir)
                    garment_dir_id = os.path.basename(GARMENT_DIR)
                    garment_ids = garment_dir_id.split(“_”) # extract top_id, bottom_id
                    top_id, bottom_id = (garment_ids[0]+“_”+garment_ids[1]), (garment_ids[2]+“_”+garment_ids[3])
                    fit_name = top_id + “_” + bottom_id
                    target_top_base_id = top_id
                    target_bottom_base_id = bottom_id
                    info = garment_info_json(os.path.join(Garment_info_path, f”{type}_{path}_transformed_data.json”))
                    combined_list = info[garment_num]
                    if not target_top_base_id == target_bottom_base_id:
                        # {aa}_transformed_data.json #train test valid
                        matching_item = next(
                            (item for item in combined_list
                            if item.get(“top_base_id”) == target_top_base_id and item.get(“bottom_base_id”) == target_bottom_base_id),
                            None
                        )
                        top_panel = matching_item[“top_panel_count”]
                        bottom_panel = matching_item[“bottom_panel_count”]
                        combi_num = 2
                    else:
                        #outfit_{aa}_transformed_data.json #train test valid
                        matching_item = next(
                            (item for item in combined_list
                            if item.get(“outfit_base_id”) == target_top_base_id),
                            None
                        )
                        outfit_panel = matching_item[“outfit_panel_count”]
                        combi_num = 1
                    fabric_path_1, fabric_path_2 = random_fabric_path(FABRIC_PATH)
                    if path.startswith(“.DS_Store”):
                        continue
                    export_zpac = os.path.join(GARMENT_DIR, f”{fit_name}.zpac”)
                    JSON_PATH = glob.glob(os.path.join(GARMENT_DIR, “Clo_Garment_*.json”))
                    STITCH_PATH = glob.glob(os.path.join(GARMENT_DIR, “Clo_Stitch_*.json”))
                    #print(STITCH_PATH)
                    stitch_json = open_json(STITCH_PATH)
                    import_api.ImportFile(avtfile_path)
                    pattern_api.ImportPatternJSON(JSON_PATH[0]) # garment import
                    export_api.ExportZPac(export_zpac)
                    import_api.ImportFile(avtfile_path)
                    import_api.ImportFile(export_zpac)
                #        import_api.ImportFile(export_zpac, add_option)
                    fabric_api.AddFabric(fabric_path_1)
                    fabric_api.AddFabric(fabric_path_2)
                    # ???? fabirc? pattern? ?????
                    if combi_num == 2 :
                        adding_top_bottom_fabric(top_panel, bottom_panel)
                    if combi_num == 1 :
                        adding_outfit_fabric(outfit_panel)
                    else :
                        print(“garment_id_list is not 1 or 2 ????.“)
                    export_api.ExportSnapshot3D(os.path.join(GARMENT_DIR,f”{fit_name}_before_drape.png”))
                    export_api.ExportOBJ(os.path.join(GARMENT_DIR,f”{fit_name}_before_drape.obj”),ieo_option)
                    utility_api.Simulate(200)
                    export_api.ExportSnapshot3D(os.path.join(GARMENT_DIR,f”{fit_name}_after_drape.png”))
                    export_api.ExportOBJ(os.path.join(GARMENT_DIR,f”{fit_name}_after_drape.obj”), ieo_option)
                    utility_api.NewProject()
                    import_api.ImportFile(avtfile_path)
                    import_api.ImportFile(export_zpac)
                    fabric_api.AddFabric(fabric_path_1)
                    fabric_api.AddFabric(fabric_path_2)
                    # ???? fabirc? pattern? ?????
                    if combi_num == 2 :
                        adding_top_bottom_fabric(top_panel, bottom_panel)
                    if combi_num == 1 :
                        adding_outfit_fabric(outfit_panel)
                    else :
                        print(“garment_id_list is not 1 or 2 ????.“)
                    pose = os.path.basename(pose_path)
                    utility_api.Simulate(200)
                    import_api.ImportPose(pose_path)
                    export_api.ExportOBJ(os.path.join(GARMENT_DIR,f”{fit_name}_{pose}_after_pose.obj”),ieo_option)
                    for custom_point in os.listdir(custom_point_path):
                        custom_point_file = os.path.join(custom_point_path, custom_point)
                        import_api.ImportFile(custom_point_file) # view import
                        export_api.ExportRenderingImage(os.path.join(GARMENT_DIR,f”{pose}_{fit_name}_{custom_point}_rendering.png”))
                utility_api.NewProject()

In [14]:
import random

ll = list(range(10))

random.choice(ll)


0